In [ ]:
%pip install -r requirements.txt

In [ ]:
from app import db, create_app
from app.models import Store

app = create_app()

In [ ]:
new_store = Store(
    name="Tesco",
    address="123 Market St",
    logo="http://example.com/logo.png",
    floor_plan="floor_plan.svg",
)

with app.app_context():
    db.session.add(new_store)
    db.session.commit()

In [1]:
with app.app_context():
    # get all stores
    stores = Store.query.all()
    print(stores)

[<Store 0e35e6fd-5e2a-4988-a95c-aa370e5b33b3>]


In [ ]:
# load category_sections.json

import json
from app.models import Shelf

store = stores[0]

with app.app_context(), open('category_sections.json') as f:
    data = json.load(f)

    for section in data:
        shelf = Shelf(
            subcategory_name=section["category"],
            section=section["division"],
            shelf_number=section["shelf"],
            store_id=store.id
        )

        db.session.add(shelf)
    db.session.commit()

In [2]:
from app.models import Shelf

with app.app_context():
    # get all stores
    shelves = Shelf.query.all()
    shelf_map = {shelf.subcategory_name: shelf.id for shelf in shelves}
    
    print(shelves[0].products)

[]


In [5]:
import pandas as pd
import numpy as np

from app import db
from app.models import Product, Shelf

# load data
data = pd.read_csv('app/tesco_groceries_dataset.csv')

# put data in products

with app.app_context():
    for index, row in data.iterrows():
        shelf_name = row['breadcrumbs'].split('~')[-2]
        shelf_id = shelf_map[shelf_name]
        
        shelf = Shelf.query.get(shelf_id)
        
        p = Product(
            name=row["name"],
            price=row["price"],
#             description=row["description"],
            image=row["images"].split('~')[0]
        )
        
        p.shelves.append(Shelf.query.get(shelf_id))
        db.session.add(p)
            
    db.session.commit()
    print("products seeded")


/var/folders/_j/w3zq6ycj0sqbl49tfqth4wq80000gn/T/ipykernel_38956/1191359281.py:17: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  shelf = Shelf.query.get(shelf_id)
/var/folders/_j/w3zq6ycj0sqbl49tfqth4wq80000gn/T/ipykernel_38956/1191359281.py:26: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  p.shelves.append(Shelf.query.get(shelf_id))


products seeded


In [7]:
from app.models import Shelf

with app.app_context():
    shelves = Shelf.query.all()
    products = shelves[0].products
    print(products[0].name)

Kiddylicious Smoothie Melts Mango 6G
